## 准备数据

In [21]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [22]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [23]:
class myModel:
    def __init__(self):
        self.W1 = tf.Variable(tf.random.normal([784, 256]), name='W1')
        self.b1 = tf.Variable(tf.zeros([256]), name='b1')
        self.W2 = tf.Variable(tf.random.normal([256, 128]), name='W2')
        self.b2 = tf.Variable(tf.zeros([128]), name='b2')
        self.W3 = tf.Variable(tf.random.normal([128, 10]), name='W3')
        self.b3 = tf.Variable(tf.zeros([10]), name='b3')

    def __call__(self, x):
        x = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        h2 = tf.nn.relu(tf.matmul(h1, self.W2) + self.b2)
        logits = tf.matmul(h2, self.W3) + self.b3
        return logits

        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [24]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [25]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1368.1594 ; accuracy 0.1296
epoch 1 : loss 845.3566 ; accuracy 0.14875
epoch 2 : loss 551.2587 ; accuracy 0.20253333
epoch 3 : loss 433.83875 ; accuracy 0.25523335
epoch 4 : loss 362.97046 ; accuracy 0.29075
epoch 5 : loss 310.92532 ; accuracy 0.32076666
epoch 6 : loss 273.21753 ; accuracy 0.35651666
epoch 7 : loss 246.18806 ; accuracy 0.3862
epoch 8 : loss 225.52943 ; accuracy 0.41221666
epoch 9 : loss 208.96904 ; accuracy 0.43476668
epoch 10 : loss 195.12933 ; accuracy 0.45425
epoch 11 : loss 183.306 ; accuracy 0.47151667
epoch 12 : loss 173.07709 ; accuracy 0.48831666
epoch 13 : loss 164.1472 ; accuracy 0.5031833
epoch 14 : loss 156.246 ; accuracy 0.5171
epoch 15 : loss 149.22844 ; accuracy 0.5291833
epoch 16 : loss 142.93399 ; accuracy 0.5412167
epoch 17 : loss 137.28781 ; accuracy 0.5531
epoch 18 : loss 132.20041 ; accuracy 0.563
epoch 19 : loss 127.57341 ; accuracy 0.57243335
epoch 20 : loss 123.34308 ; accuracy 0.5815833
epoch 21 : loss 119.466965 ; accuracy 0.589